In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [ ]:
%%sql
select 'hola' as col

In [ ]:
%%sql

create or replace table competencia_02 as 

select 
    *
from read_csv_auto('~/buckets/b1/competencia_02_crudo.csv.gz')

In [ ]:
%sql select count(*) from competencia_02

In [ ]:
%%sql

create or replace table competencia_02 as 

select 
    *,
    strptime(foto_mes, '%Y%m') as foto_mes_ts
from competencia_02

In [ ]:
duckdb.query("describe select foto_mes_ts from competencia_02")

In [ ]:
%%sql 

select 
* 
from competencia_02 
limit 5

In [ ]:
%%sql

select 
    foto_mes, 
    count(*) as cantidad                        
from competencia_02
group by foto_mes

In [ ]:
%%sql

select 
    max(foto_mes_ts)                      
from competencia_02

In [ ]:
%%sql

create or replace table competencia_02 as 

select
    *,
    lead(t0.foto_mes_ts, 1) over ventana as foto_mes_ts_prox_1, 
    lead(t0.foto_mes_ts, 2) over ventana as foto_mes_ts_prox_2
from competencia_02 t0
window ventana as (partition by numero_de_cliente order by foto_mes)

In [ ]:
%%sql

select 
    *
from competencia_02
limit 5

In [ ]:
%%sql

create or replace table competencia_02 as 

select
    *,
    case
        WHEN (date_diff('month', t0.foto_mes_ts, TIMESTAMP '2021-05-01') <= 1) THEN NULL
        WHEN ((t0.foto_mes_ts_prox_1 is NULL) OR 
              (date_diff('month', t0.foto_mes_ts, t0.foto_mes_ts_prox_1) != 1)) THEN 'BAJA+1'
        WHEN ((t0.foto_mes_ts_prox_2 is NULL) OR 
              (date_diff('month', t0.foto_mes_ts, t0.foto_mes_ts_prox_2) != 2)) THEN 'BAJA+2' 
        ELSE 'CONTINUA'
    END as clase_ternaria
from competencia_02 t0

In [ ]:
%%sql

ALTER TABLE competencia_02 DROP foto_mes_ts;
ALTER TABLE competencia_02 DROP foto_mes_ts_prox_1;
ALTER TABLE competencia_02 DROP foto_mes_ts_prox_2;

In [ ]:
%sql select * from competencia_02 limit 5

In [ ]:
%%sql

select 
    clase_ternaria, 
    count(*) as cant,
from competencia_02
group by clase_ternaria

In [ ]:
%%sql
COPY competencia_02 TO '~/buckets/b1/competencia_02.csv.gz' (FORMAT CSV, COMPRESSION GZIP, HEADER)